In [1]:
from scipy.stats import qmc
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import shutil
import os
from sklearn.metrics import mean_squared_error
from math import sqrt
import subprocess
import scienceplots

import warnings
import matplotlib.pyplot as plt
import random

warnings.filterwarnings("ignore")

## Calibration strategy:
- Calibrate on both june and july
- Leave out 10-20% 
- Each sensor within each week is its own dataset
- Leave out a random sensor from calibration
- Consider all sensors as data set (22 sensors) remove 20%, then run calibration to minimize rmse, Then run validation on the sensors that were left out for both week in june and july
- Leave latin hypercube widths ranges pretty wide, run 5000 times for both validation sets



In [2]:
#reading in data with relative paths so that it can be used by others who clone repository from github
script_dir = os.getcwd()

#Calibration Folder
calibration_folder_path = os.path.abspath(os.path.join(script_dir, '..', '..', 'calibration'))

#July Source Folder
July_calibration_path = os.path.abspath(os.path.join(script_dir, '..', '..', 'calibration', 'July'))

#June Source Folder
June_calibration_path = os.path.abspath(os.path.join(script_dir, '..', '..', 'calibration', 'June'))

#August Source Folder
August_calibration_path = os.path.abspath(os.path.join(script_dir, '..', '..', 'calibration', 'August'))

#Temperature Observations folder
observed_temp_path = os.path.abspath(os.path.join(script_dir, '..', '..', 'observations', 'TemperatureDataObs'))


In [4]:
# June hypercube results
june_cal = pd.read_csv(os.path.join(June_calibration_path, 'OptRMSE1000_June.csv'))

#July Hypercube results
july_cal = pd.read_csv(os.path.join(July_calibration_path, 'OptRMSE1000_July.csv'))

#August Hypercube results
August_cal = pd.read_csv(os.path.join(August_calibration_path, 'OptRMSE1000_August.csv'))

july_cal.sort_values(by = 'overall rmse')


,thermal conductivity,thermal diffusivity,hypo thickness,hypo %,sediment porosity,wind function a,wind function b,accretion temp,tree canopy,grass canopy,...,2.3,12.3,2.7,8.6,15.0,8.0,13.9,5.7,overall rmse,nse
528,1.508749,0.006725,0.276348,0.015875,0.885097,2.691316e-09,1.156092e-09,9.187339,0.709196,0.193165,...,0.577302,0.730052,0.591435,0.827880,0.647226,0.650736,0.639423,0.719886,0.634760,0.905352
565,1.514415,0.006435,0.278930,0.012998,0.924070,2.975997e-09,1.085697e-09,10.168973,0.790460,0.136833,...,0.605021,0.674989,0.633783,0.845096,0.679703,0.657145,0.627849,0.732747,0.639246,0.904000
976,1.556688,0.006385,0.262875,0.011500,0.886685,2.410471e-09,1.078720e-09,9.029475,0.735846,0.178853,...,0.632326,0.776500,0.552547,0.845831,0.559831,0.687305,0.647811,0.803838,0.643127,0.902195
412,1.584759,0.006540,0.262099,0.016477,0.719212,3.442137e-09,1.070972e-09,9.212213,0.542557,0.120450,...,0.636113,0.783706,0.620026,0.847763,0.577410,0.654402,0.669742,0.733797,0.646674,0.901583
998,1.441378,0.006238,0.260855,0.020295,0.973506,3.217560e-09,1.227746e-09,10.998451,0.637334,0.120816,...,0.624730,0.776608,0.586430,0.845150,0.643207,0.659545,0.667689,0.722731,0.646812,0.901465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,1.524855,0.006501,0.363796,0.016705,0.656201,3.423447e-09,3.792494e-09,10.279253,0.777645,0.131904,...,1.260537,0.985274,1.402699,1.460400,1.243881,1.019666,1.076771,1.147262,1.111992,0.699503
121,1.568117,0.006608,0.381000,0.028738,0.835628,1.392913e-09,1.073711e-09,12.647921,0.524493,0.130066,...,1.347828,1.489093,1.071318,1.489949,0.670547,1.154164,1.133922,1.328484,1.113549,0.692597
197,1.435729,0.006733,0.397202,0.028513,0.878312,3.993035e-09,3.473485e-09,9.982615,0.790834,0.153215,...,1.263535,1.002031,1.403929,1.433548,1.348265,0.978819,1.128620,1.088742,1.115903,0.696887
883,1.519927,0.006327,0.337271,0.019783,0.997276,1.288786e-09,1.117870e-09,12.589185,0.516664,0.197886,...,1.395112,1.504368,1.122251,1.524730,0.603795,1.187913,1.120828,1.393594,1.134521,0.678605


In [5]:
lhs = pd.read_csv(os.path.join(calibration_folder_path, 'hypercube_parameters1000.csv'))
lhs

,Unnamed: 0,thermal conductivity,thermal diffusivity,hypo thickness,hypo %,sediment porosity,wind function a,wind function b,accretion temp,tree canopy,grass canopy,tree overhang,grass overhang
0,0,1.589739,0.006693,0.277969,0.012962,0.607604,2.700286e-09,2.792647e-09,10.830394,0.559755,0.108751,1.530267,0.826332
1,1,1.593486,0.006308,0.386867,0.012137,0.646473,3.447290e-09,3.282401e-09,9.383360,0.758393,0.144381,1.231513,0.398736
2,2,1.447345,0.006293,0.305921,0.034472,0.650536,1.089236e-09,2.251392e-09,12.565471,0.594054,0.184032,0.335779,0.052414
3,3,1.406989,0.006761,0.383032,0.020680,0.873685,2.820607e-09,1.102571e-09,10.007236,0.751449,0.183003,0.301451,0.196670
4,4,1.570616,0.006592,0.393133,0.035451,0.625397,2.270726e-09,1.609290e-09,12.030047,0.696049,0.157197,0.638785,0.205271
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,1.567029,0.006743,0.283044,0.013645,0.927248,2.399722e-09,1.637229e-09,9.165179,0.629259,0.160541,0.552024,0.739461
996,996,1.563420,0.006584,0.258387,0.021050,0.685840,1.061351e-09,2.431400e-09,11.579462,0.599685,0.135954,1.639880,0.487997
997,997,1.450943,0.006686,0.389896,0.023276,0.992971,2.260998e-09,2.914381e-09,9.810255,0.593766,0.153070,0.418899,0.693607
998,998,1.441378,0.006238,0.260855,0.020295,0.973506,3.217560e-09,1.227746e-09,10.998451,0.637334,0.120816,0.842850,0.466131


## Calibration/Validation split

In [45]:
july_sensors

['0.0',
 '18.5',
 '17.6',
 '16.2',
 '7.2',
 '10.8',
 '2.3',
 '12.3',
 '2.7',
 '8.6',
 '15.0',
 '8.0',
 '13.9',
 '5.7']

In [49]:
#July

#get july sensor names, and choose 30% of sensors to run calibration on
july_calibration = pd.read_csv(os.path.join(July_calibration_path, 'OptRMSE1000_July.csv'))
#select the columns corresponding to sensors
july_sensors = july_calibration.columns[13:27].tolist()
july_sensors.remove('18.5')
#find what 30% of sensor count is
july_select30 = int(len(july_sensors) * 0.3)
#subset sensors to select 30%
july30_subset = random.sample(july_sensors, july_select30)
#calculate overall rmse for all of the columns that dont include those left out in the split
july_columns_to_keep = [col for col in july_sensors if col not in july30_subset]
july_calibration_df = july_calibration[july_columns_to_keep]
july_calibration_df = july_calibration_df.add_suffix('_julyCAL')

#calculate rmse for the validation sensors
july_validation_df = july_calibration[july30_subset]
july_validation_df = july_validation_df.add_suffix('_julyVAL')
july_validation_df



,2.7_julyVAL,5.7_julyVAL,2.3_julyVAL
0,0.783275,0.863509,0.777907
1,1.326570,1.133810,1.194392
2,0.908136,1.159545,1.148923
3,0.572614,0.725806,0.643124
4,0.631099,0.802580,0.803318
...,...,...,...
995,0.588347,0.808691,0.679793
996,0.740819,1.047305,0.949373
997,0.737247,0.797149,0.778457
998,0.586430,0.722731,0.624730


In [50]:
#June

#get july sensor names, and choose 30% of sensors to run calibration on
june_calibration = pd.read_csv(os.path.join(June_calibration_path, 'OptRMSE1000_June.csv'))
#select the columns corresponding to sensors
june_sensors = june_calibration.columns[13:21].tolist()
june_sensors.remove('18.5')
#find what 30% of sensor count is
june_select30 = int(len(june_sensors) * 0.3)
#subset sensors to select 30%
june30_subset = random.sample(june_sensors, june_select30)
#calculate overall rmse for all of the columns that dont include those left out in the split
june_columns_to_keep = [col for col in june_sensors if col not in june30_subset]
june_calibration_df = june_calibration[june_columns_to_keep]
june_calibration_df = june_calibration_df.add_suffix('_juneCAL')

#calculate rmse for the validation sensors
june_validation_df = june_calibration[june30_subset]
june_validation_df = june_validation_df.add_suffix('_juneVAL')
june_validation_df


,10.8_juneVAL,7.2_juneVAL
0,0.662992,0.491326
1,0.922832,0.820840
2,1.261451,0.807704
3,0.856215,0.526663
4,1.037009,0.565699
...,...,...
995,0.802608,0.519408
996,1.109380,0.692762
997,0.780624,0.434756
998,0.822487,0.510175


In [51]:
#August

#get july sensor names, and choose 30% of sensors to run calibration on
august_calibration = pd.read_csv(os.path.join(August_calibration_path, 'OptRMSE1000_August.csv'))
#select the columns corresponding to sensors
august_sensors = august_calibration.columns[13:21].tolist()
august_sensors.remove('18.5')
#find what 30% of sensor count is
august_select30 = int(len(august_sensors) * 0.3)
#subset sensors to select 30%
august30_subset = random.sample(august_sensors, august_select30)
#calculate overall rmse for all of the columns that dont include those left out in the split
august_columns_to_keep = [col for col in august_sensors if col not in august30_subset]
august_calibration_df = august_calibration[august_columns_to_keep]
august_calibration_df = august_calibration_df.add_suffix('_augCAL')

#calculate rmse for the validation sensors
august_validation_df = august_calibration[august30_subset]
august_validation_df = august_validation_df.add_suffix('_augVAL')
august_sensors

['0.0', '17.6', '7.2', '2.3', '12.3', '2.7', '8.6']

In [52]:
#Merging june and July validation metrics

validation_merge = pd.merge(july_validation_df, july_validation_df, left_index = True, right_index = True)
validation_merge = pd.merge(validation_merge, august_validation_df, left_index = True, right_index = True)
validation_merge = pd.merge(lhs, validation_merge, left_index = True, right_index = True)
validation_merge

,Unnamed: 0,thermal conductivity,thermal diffusivity,hypo thickness,hypo %,sediment porosity,wind function a,wind function b,accretion temp,tree canopy,...,tree overhang,grass overhang,2.7_julyVAL_x,5.7_julyVAL_x,2.3_julyVAL_x,2.7_julyVAL_y,5.7_julyVAL_y,2.3_julyVAL_y,7.2_augVAL,2.7_augVAL
0,0,1.589739,0.006693,0.277969,0.012962,0.607604,2.700286e-09,2.792647e-09,10.830394,0.559755,...,1.530267,0.826332,0.783275,0.863509,0.777907,0.783275,0.863509,0.777907,0.557973,0.568583
1,1,1.593486,0.006308,0.386867,0.012137,0.646473,3.447290e-09,3.282401e-09,9.383360,0.758393,...,1.231513,0.398736,1.326570,1.133810,1.194392,1.326570,1.133810,1.194392,0.729840,1.093974
2,2,1.447345,0.006293,0.305921,0.034472,0.650536,1.089236e-09,2.251392e-09,12.565471,0.594054,...,0.335779,0.052414,0.908136,1.159545,1.148923,0.908136,1.159545,1.148923,1.077245,1.014872
3,3,1.406989,0.006761,0.383032,0.020680,0.873685,2.820607e-09,1.102571e-09,10.007236,0.751449,...,0.301451,0.196670,0.572614,0.725806,0.643124,0.572614,0.725806,0.643124,0.556467,0.476709
4,4,1.570616,0.006592,0.393133,0.035451,0.625397,2.270726e-09,1.609290e-09,12.030047,0.696049,...,0.638785,0.205271,0.631099,0.802580,0.803318,0.631099,0.802580,0.803318,0.660051,0.588159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,1.567029,0.006743,0.283044,0.013645,0.927248,2.399722e-09,1.637229e-09,9.165179,0.629259,...,0.552024,0.739461,0.588347,0.808691,0.679793,0.588347,0.808691,0.679793,0.629600,0.528681
996,996,1.563420,0.006584,0.258387,0.021050,0.685840,1.061351e-09,2.431400e-09,11.579462,0.599685,...,1.639880,0.487997,0.740819,1.047305,0.949373,0.740819,1.047305,0.949373,0.938167,0.821412
997,997,1.450943,0.006686,0.389896,0.023276,0.992971,2.260998e-09,2.914381e-09,9.810255,0.593766,...,0.418899,0.693607,0.737247,0.797149,0.778457,0.737247,0.797149,0.778457,0.470116,0.518702
998,998,1.441378,0.006238,0.260855,0.020295,0.973506,3.217560e-09,1.227746e-09,10.998451,0.637334,...,0.842850,0.466131,0.586430,0.722731,0.624730,0.586430,0.722731,0.624730,0.624833,0.546392


In [53]:
#Merging june and July calibration metrics

calibration_merge = pd.merge(june_calibration_df, july_calibration_df, left_index = True, right_index = True)
calibration_merge = pd.merge(calibration_merge, august_calibration_df, left_index = True, right_index = True)
calibration_merge = pd.merge(lhs, calibration_merge, left_index = True, right_index = True)
calibration_merge

,Unnamed: 0,thermal conductivity,thermal diffusivity,hypo thickness,hypo %,sediment porosity,wind function a,wind function b,accretion temp,tree canopy,...,12.3_julyCAL,8.6_julyCAL,15.0_julyCAL,8.0_julyCAL,13.9_julyCAL,0.0_augCAL,17.6_augCAL,2.3_augCAL,12.3_augCAL,8.6_augCAL
0,0,1.589739,0.006693,0.277969,0.012962,0.607604,2.700286e-09,2.792647e-09,10.830394,0.559755,...,0.848957,0.993712,0.730263,0.751217,0.777860,0.661590,0.447542,0.644771,0.852514,0.902369
1,1,1.593486,0.006308,0.386867,0.012137,0.646473,3.447290e-09,3.282401e-09,9.383360,0.758393,...,0.914494,1.400414,1.109868,0.990938,0.987615,1.110089,0.426023,1.169445,0.678209,1.173263
2,2,1.447345,0.006293,0.305921,0.034472,0.650536,1.089236e-09,2.251392e-09,12.565471,0.594054,...,1.313728,1.312711,0.751848,1.021776,1.031166,1.138036,0.478709,1.051571,1.492357,1.501667
3,3,1.406989,0.006761,0.383032,0.020680,0.873685,2.820607e-09,1.102571e-09,10.007236,0.751449,...,0.769887,0.848417,0.698567,0.653467,0.710575,0.601421,0.443436,0.557297,0.850348,0.890421
4,4,1.570616,0.006592,0.393133,0.035451,0.625397,2.270726e-09,1.609290e-09,12.030047,0.696049,...,0.992500,0.942368,0.852984,0.720781,0.911980,0.697229,0.434057,0.683591,1.093377,1.030750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,1.567029,0.006743,0.283044,0.013645,0.927248,2.399722e-09,1.637229e-09,9.165179,0.629259,...,0.835594,0.874858,0.576977,0.694241,0.698746,0.642891,0.472088,0.574997,0.956207,0.934958
996,996,1.563420,0.006584,0.258387,0.021050,0.685840,1.061351e-09,2.431400e-09,11.579462,0.599685,...,1.176226,1.155921,0.655427,0.919288,0.912974,0.932529,0.477365,0.838539,1.332398,1.310552
997,997,1.450943,0.006686,0.389896,0.023276,0.992971,2.260998e-09,2.914381e-09,9.810255,0.593766,...,0.889647,0.950183,0.822349,0.702180,0.844186,0.602282,0.422950,0.630727,0.863802,0.840453
998,998,1.441378,0.006238,0.260855,0.020295,0.973506,3.217560e-09,1.227746e-09,10.998451,0.637334,...,0.776608,0.845150,0.643207,0.659545,0.667689,0.644363,0.440316,0.574437,0.923622,0.929759


In [54]:
calibration_merge.columns[13::]

Index(['0.0_juneCAL', '17.6_juneCAL', '16.2_juneCAL', '2.4_juneCAL',
       '12.3_juneCAL', '0.0_julyCAL', '17.6_julyCAL', '16.2_julyCAL',
       '7.2_julyCAL', '10.8_julyCAL', '12.3_julyCAL', '8.6_julyCAL',
       '15.0_julyCAL', '8.0_julyCAL', '13.9_julyCAL', '0.0_augCAL',
       '17.6_augCAL', '2.3_augCAL', '12.3_augCAL', '8.6_augCAL'],
      dtype='object')

In [55]:
calibration_merge['calibration overall rmse'] = calibration_merge[calibration_merge.columns[13::]].mean(axis=1)
calibration_merge

,Unnamed: 0,thermal conductivity,thermal diffusivity,hypo thickness,hypo %,sediment porosity,wind function a,wind function b,accretion temp,tree canopy,...,8.6_julyCAL,15.0_julyCAL,8.0_julyCAL,13.9_julyCAL,0.0_augCAL,17.6_augCAL,2.3_augCAL,12.3_augCAL,8.6_augCAL,calibration overall rmse
0,0,1.589739,0.006693,0.277969,0.012962,0.607604,2.700286e-09,2.792647e-09,10.830394,0.559755,...,0.993712,0.730263,0.751217,0.777860,0.661590,0.447542,0.644771,0.852514,0.902369,0.688041
1,1,1.593486,0.006308,0.386867,0.012137,0.646473,3.447290e-09,3.282401e-09,9.383360,0.758393,...,1.400414,1.109868,0.990938,0.987615,1.110089,0.426023,1.169445,0.678209,1.173263,0.928529
2,2,1.447345,0.006293,0.305921,0.034472,0.650536,1.089236e-09,2.251392e-09,12.565471,0.594054,...,1.312711,0.751848,1.021776,1.031166,1.138036,0.478709,1.051571,1.492357,1.501667,0.993031
3,3,1.406989,0.006761,0.383032,0.020680,0.873685,2.820607e-09,1.102571e-09,10.007236,0.751449,...,0.848417,0.698567,0.653467,0.710575,0.601421,0.443436,0.557297,0.850348,0.890421,0.645802
4,4,1.570616,0.006592,0.393133,0.035451,0.625397,2.270726e-09,1.609290e-09,12.030047,0.696049,...,0.942368,0.852984,0.720781,0.911980,0.697229,0.434057,0.683591,1.093377,1.030750,0.764373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,1.567029,0.006743,0.283044,0.013645,0.927248,2.399722e-09,1.637229e-09,9.165179,0.629259,...,0.874858,0.576977,0.694241,0.698746,0.642891,0.472088,0.574997,0.956207,0.934958,0.659286
996,996,1.563420,0.006584,0.258387,0.021050,0.685840,1.061351e-09,2.431400e-09,11.579462,0.599685,...,1.155921,0.655427,0.919288,0.912974,0.932529,0.477365,0.838539,1.332398,1.310552,0.868413
997,997,1.450943,0.006686,0.389896,0.023276,0.992971,2.260998e-09,2.914381e-09,9.810255,0.593766,...,0.950183,0.822349,0.702180,0.844186,0.602282,0.422950,0.630727,0.863802,0.840453,0.693350
998,998,1.441378,0.006238,0.260855,0.020295,0.973506,3.217560e-09,1.227746e-09,10.998451,0.637334,...,0.845150,0.643207,0.659545,0.667689,0.644363,0.440316,0.574437,0.923622,0.929759,0.647960


In [58]:
validation_merge['validation overall rmse'] = validation_merge[validation_merge.columns[13::]].mean(axis=1)
validation_merge

,Unnamed: 0,thermal conductivity,thermal diffusivity,hypo thickness,hypo %,sediment porosity,wind function a,wind function b,accretion temp,tree canopy,...,grass overhang,2.7_julyVAL_x,5.7_julyVAL_x,2.3_julyVAL_x,2.7_julyVAL_y,5.7_julyVAL_y,2.3_julyVAL_y,7.2_augVAL,2.7_augVAL,validation overall rmse
0,0,1.589739,0.006693,0.277969,0.012962,0.607604,2.700286e-09,2.792647e-09,10.830394,0.559755,...,0.826332,0.783275,0.863509,0.777907,0.783275,0.863509,0.777907,0.557973,0.568583,0.746992
1,1,1.593486,0.006308,0.386867,0.012137,0.646473,3.447290e-09,3.282401e-09,9.383360,0.758393,...,0.398736,1.326570,1.133810,1.194392,1.326570,1.133810,1.194392,0.729840,1.093974,1.141670
2,2,1.447345,0.006293,0.305921,0.034472,0.650536,1.089236e-09,2.251392e-09,12.565471,0.594054,...,0.052414,0.908136,1.159545,1.148923,0.908136,1.159545,1.148923,1.077245,1.014872,1.065666
3,3,1.406989,0.006761,0.383032,0.020680,0.873685,2.820607e-09,1.102571e-09,10.007236,0.751449,...,0.196670,0.572614,0.725806,0.643124,0.572614,0.725806,0.643124,0.556467,0.476709,0.614533
4,4,1.570616,0.006592,0.393133,0.035451,0.625397,2.270726e-09,1.609290e-09,12.030047,0.696049,...,0.205271,0.631099,0.802580,0.803318,0.631099,0.802580,0.803318,0.660051,0.588159,0.715276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,1.567029,0.006743,0.283044,0.013645,0.927248,2.399722e-09,1.637229e-09,9.165179,0.629259,...,0.739461,0.588347,0.808691,0.679793,0.588347,0.808691,0.679793,0.629600,0.528681,0.663993
996,996,1.563420,0.006584,0.258387,0.021050,0.685840,1.061351e-09,2.431400e-09,11.579462,0.599685,...,0.487997,0.740819,1.047305,0.949373,0.740819,1.047305,0.949373,0.938167,0.821412,0.904322
997,997,1.450943,0.006686,0.389896,0.023276,0.992971,2.260998e-09,2.914381e-09,9.810255,0.593766,...,0.693607,0.737247,0.797149,0.778457,0.737247,0.797149,0.778457,0.470116,0.518702,0.701816
998,998,1.441378,0.006238,0.260855,0.020295,0.973506,3.217560e-09,1.227746e-09,10.998451,0.637334,...,0.466131,0.586430,0.722731,0.624730,0.586430,0.722731,0.624730,0.624833,0.546392,0.629876


In [61]:
cal_sort = validation_merge.sort_values(by = 'validation overall rmse')
best_cal_params = cal_sort.iloc[0]['Unnamed: 0']
best_cal_params

389.0

In [62]:
# cal_values = calibration_merge.loc(calibration_merge['Unnamed: 0'] == calibration_merge)
validation_values = calibration_merge.loc[calibration_merge['Unnamed: 0'] == best_cal_params]
validation_values

,Unnamed: 0,thermal conductivity,thermal diffusivity,hypo thickness,hypo %,sediment porosity,wind function a,wind function b,accretion temp,tree canopy,...,8.6_julyCAL,15.0_julyCAL,8.0_julyCAL,13.9_julyCAL,0.0_augCAL,17.6_augCAL,2.3_augCAL,12.3_augCAL,8.6_augCAL,calibration overall rmse
389,389,1.58661,0.006799,0.315955,0.039207,0.973074,2.765804e-09,1.382219e-09,9.171916,0.693968,...,0.818748,0.889651,0.604366,0.774671,0.578791,0.389956,0.587875,0.847759,0.824583,0.671186
